In [ ]:
# Install Dependencies

!pip uninstall -y numpy
!pip install --force-reinstall numpy==1.26.4

!pip install torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 sentence-transformers==2.2.2
!pip install pandas==2.0.0
!pip install transformers==4.41.0 scikit-learn==1.2.0
!pip install huggingface-hub==0.25.2
!pip install nltk==3.8.1 rouge-score==0.1.2 bert-score==0.3.13 -q
!pip install tqdm==4.66.5 -q

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 101.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB

In [ ]:
# Setup and Imports

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer,
    DPRContextEncoder, DPRQuestionEncoder,
    DPRContextEncoderTokenizer, DPRQuestionEncoderTokenizer
)
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
import os
from google.colab import drive
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import bert_score
from torch.cuda.amp import GradScaler, autocast
import torch.nn as nn
import torch.optim as optim
import string
import nltk

nltk.download('wordnet')
nltk.download('punkt')

# Mount Google Drive
drive.mount('/content/drive')

# Configuration (aligned with Steps 1 and 2)
class Config:
    BASE_PATH = "/content/drive/MyDrive/LJMU-Datasets"
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    T5_MODEL_NAME = "t5-small"
    DPR_CTX_MODEL_NAME = "facebook/dpr-ctx_encoder-single-nq-base"
    DPR_QUESTION_MODEL_NAME = "facebook/dpr-question_encoder-single-nq-base"
    BATCH_SIZE = 16
    MAX_EPOCHS = 3
    NUM_WORKERS = 4
    MAX_LENGTH = 256
    SUBSET_SIZE = 500
    HOTPOTQA_MAX_SAMPLES = 1000
    WIKIDATA_SUBSET_SIZE = 30000

CONFIG = Config()
print(f"Using device: {CONFIG.DEVICE}")

# Clear GPU memory
torch.cuda.empty_cache()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda


AttributeError: Can't get attribute 'RetrievalDataset' on <module '__main__'>

In [ ]:
# Load Artifacts from Step 2 and Create Triple Validation Set

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, DPRQuestionEncoderTokenizer, T5ForConditionalGeneration, DPRContextEncoder, DPRQuestionEncoder
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
import os

save_path = '/content/drive/MyDrive/bert_retrieval_artifacts'

# Load raw datasets instead of pickled DataLoaders
qa_train_path = os.path.join(CONFIG.BASE_PATH, "qa_train.csv")
qa_val_path = os.path.join(CONFIG.BASE_PATH, "qa_val.csv")
triple_train_path = os.path.join(CONFIG.BASE_PATH, "triple_train.csv")

qa_train_df = pd.read_csv(qa_train_path)
qa_val_df = pd.read_csv(qa_val_path)
triple_train_df = pd.read_csv(triple_train_path)

# Split triple_train_df into train and validation sets (80/20)
triple_train_df, triple_val_df = train_test_split(triple_train_df, train_size=0.8, random_state=42)
print(f"Triple Train Size: {len(triple_train_df)}, Triple Val Size: {len(triple_val_df)}")

# Load all_candidates
with open(os.path.join(save_path, 'all_candidates.pkl'), 'rb') as f:
    import pickle
    all_candidates = pickle.load(f)

# Define RetrievalDataset class
class RetrievalDataset(Dataset):
    def __init__(self, df: pd.DataFrame, t5_tokenizer: T5Tokenizer, dpr_question_tokenizer: DPRQuestionEncoderTokenizer,
                 max_length: int = 256, task: str = "qa", candidate_objects: list = None):
        self.t5_tokenizer = t5_tokenizer
        self.dpr_question_tokenizer = dpr_question_tokenizer
        self.max_length = max_length
        self.task = task
        self.data = df
        self.candidate_objects = candidate_objects

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        question = row["question"]
        context = row["context"]
        answer = row["answer"]

        if self.task == "qa":
            t5_input_text = f"answer the question exactly as in the reference: {question} context: {context} ->"
        else:
            t5_input_text = f"complete the triple with the exact object: {question} ->"
        t5_inputs = self.t5_tokenizer(
            t5_input_text,
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )
        t5_labels = self.t5_tokenizer(
            answer,
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )

        dpr_inputs = self.dpr_question_tokenizer(
            question,
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )

        item = {
            "task": self.task,
            "t5_input_ids": t5_inputs["input_ids"].squeeze(),
            "t5_attention_mask": t5_inputs["attention_mask"].squeeze(),
            "t5_labels": t5_labels["input_ids"].squeeze(),
            "dpr_input_ids": dpr_inputs["input_ids"].squeeze(),
            "dpr_attention_mask": dpr_inputs["attention_mask"].squeeze(),
            "question": question,
            "answer": answer
        }

        if self.task == "triple" and self.candidate_objects:
            label_idx = self.candidate_objects.index(answer) if answer in self.candidate_objects else -1
            item["label_idx"] = label_idx

        return item

# Create DataLoaders from raw data
t5_tokenizer = T5Tokenizer.from_pretrained(CONFIG.T5_MODEL_NAME)
dpr_question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(CONFIG.DPR_QUESTION_MODEL_NAME)

qa_train_dataset = RetrievalDataset(qa_train_df, t5_tokenizer, dpr_question_tokenizer, task="qa", candidate_objects=all_candidates)
qa_val_dataset = RetrievalDataset(qa_val_df, t5_tokenizer, dpr_question_tokenizer, task="qa", candidate_objects=all_candidates)
triple_train_dataset = RetrievalDataset(triple_train_df, t5_tokenizer, dpr_question_tokenizer, task="triple", candidate_objects=all_candidates)
triple_val_dataset = RetrievalDataset(triple_val_df, t5_tokenizer, dpr_question_tokenizer, task="triple", candidate_objects=all_candidates)

qa_train_loader = DataLoader(qa_train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True, num_workers=CONFIG.NUM_WORKERS)
qa_val_loader = DataLoader(qa_val_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False, num_workers=CONFIG.NUM_WORKERS)
triple_train_loader = DataLoader(triple_train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True, num_workers=CONFIG.NUM_WORKERS)
triple_val_loader = DataLoader(triple_val_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False, num_workers=CONFIG.NUM_WORKERS)

print(f"Created QA DataLoaders: QA Train={len(qa_train_dataset)}, QA Val={len(qa_val_dataset)}")
print(f"Created Triple DataLoaders: Triple Train={len(triple_train_dataset)}, Triple Val={len(triple_val_dataset)}")

# Load T5 model and tokenizer (for perplexity in reward function)
t5_qa_model = T5ForConditionalGeneration.from_pretrained(CONFIG.T5_MODEL_NAME).to(CONFIG.DEVICE)
t5_qa_model.load_state_dict(torch.load(os.path.join(save_path, 't5_qa_model.pt')))
t5_tokenizer = T5Tokenizer.from_pretrained(CONFIG.T5_MODEL_NAME)
t5_qa_model.eval()

# Load DPR models and tokenizers
ctx_encoder = DPRContextEncoder.from_pretrained(os.path.join(CONFIG.BASE_PATH, "dpr_ctx_encoder_final")).to(CONFIG.DEVICE)
question_encoder = DPRQuestionEncoder.from_pretrained(os.path.join(CONFIG.BASE_PATH, "dpr_question_encoder_final")).to(CONFIG.DEVICE)
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained(CONFIG.DPR_CTX_MODEL_NAME)
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(CONFIG.DPR_QUESTION_MODEL_NAME)
candidate_embeddings = torch.load(os.path.join(save_path, 'dpr_candidate_embeddings.pt')).to(CONFIG.DEVICE)

# Load sentence transformer (for cosine similarity in reward function)
sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')

print("Artifacts loaded from Step 2 and triple validation set created.")

Triple Train Size: 3999, Triple Val Size: 1000


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Artifacts loaded from Step 2 and triple validation set created.


In [ ]:
# Define Helper Functions

# Normalize text for evaluation
def normalize_text(text: str) -> str:
    text = str(text).lower().strip()
    text = text.translate(str.maketrans("", "", string.punctuation))
    articles = {'a', 'an', 'the'}
    words = text.split()
    words = [word for word in words if word not in articles]
    return ' '.join(words)

# Compute BLEU score
def compute_bleu(generated: str, reference: str) -> float:
    return sentence_bleu([reference.split()], generated.split())

# Compute ROUGE-L score
def compute_rouge_l(generated: str, reference: str) -> float:
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    return scorer.score(reference, generated)['rougeL'].fmeasure

# Compute BERTScore
def compute_bertscore(generated: str, reference: str) -> float:
    return bert_score.score([generated], [reference], lang="en", verbose=False)[2].mean().item()

# Compute perplexity using T5
def compute_perplexity(text: str) -> float:
    with torch.no_grad():
        inputs = t5_tokenizer(text, return_tensors="pt", max_length=CONFIG.MAX_LENGTH, truncation=True, padding=True)
        input_ids = inputs["input_ids"].to(CONFIG.DEVICE)
        attention_mask = inputs["attention_mask"].to(CONFIG.DEVICE)
        outputs = t5_qa_model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        perplexity = torch.exp(loss).item()
    return perplexity

# Compute cosine similarity using sentence transformer
def compute_cosine_similarity(text1: str, text2: str) -> float:
    embeddings = sentence_transformer.encode([text1, text2], batch_size=32, show_progress_bar=False)
    cos_sim = np.dot(embeddings[0], embeddings[1]) / (np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1]))
    return cos_sim

print("Helper functions defined.")

Helper functions defined.


In [ ]:
# Define RL Reward Function

def compute_reward(generated: str, reference: str) -> float:
    """
    Compute a composite reward based on BLEU, cosine similarity, and perplexity.
    """
    # Normalize texts
    gen = normalize_text(generated)
    ref = normalize_text(reference)

    # Compute BLEU score (exactness)
    bleu = compute_bleu(gen, ref)
    print(f"BLEU Score: {bleu:.4f}")

    # Compute cosine similarity (semantic similarity)
    cos_sim = compute_cosine_similarity(gen, ref)
    print(f"Cosine Similarity: {cos_sim:.4f}")

    # Compute perplexity (fluency)
    perplexity = compute_perplexity(gen)
    # Lower perplexity is better, so invert and normalize
    perplexity_score = 1.0 / (1.0 + perplexity)
    print(f"Perplexity Score: {perplexity_score:.4f}")

    # Composite reward (weighted combination)
    reward = 0.4 * bleu + 0.4 * cos_sim + 0.2 * perplexity_score
    print(f"Composite Reward: {reward:.4f}")
    return reward

# Test reward function on a few examples from QA and triple data
print("Testing reward function on QA data...")
val_batch = next(iter(qa_val_loader))
test_inputs = val_batch["t5_input_ids"][:5].to(CONFIG.DEVICE)
test_attention_mask = val_batch["t5_attention_mask"][:5].to(CONFIG.DEVICE)
test_references = val_batch["answer"][:5]

with torch.no_grad():
    generated_ids = t5_qa_model.generate(
        input_ids=test_inputs,
        attention_mask=test_attention_mask,
        max_new_tokens=100,
        num_beams=20,
        temperature=0.5,
        no_repeat_ngram_size=2
    )
    generated_texts = [t5_tokenizer.decode(g_ids, skip_special_tokens=True).lower().strip() for g_ids in generated_ids]

for gen, ref in zip(generated_texts, test_references):
    print(f"Generated: {gen}")
    print(f"Reference: {ref}")
    reward = compute_reward(gen, ref)
    print(f"Reward: {reward:.4f}\n")

print("Testing reward function on triple data...")
val_batch = next(iter(triple_val_loader))
test_inputs = val_batch["t5_input_ids"][:5].to(CONFIG.DEVICE)
test_attention_mask = val_batch["t5_attention_mask"][:5].to(CONFIG.DEVICE)
test_references = val_batch["answer"][:5]

with torch.no_grad():
    generated_ids = t5_qa_model.generate(
        input_ids=test_inputs,
        attention_mask=test_attention_mask,
        max_new_tokens=100,
        num_beams=20,
        temperature=0.5,
        no_repeat_ngram_size=2
    )
    generated_texts = [t5_tokenizer.decode(g_ids, skip_special_tokens=True).lower().strip() for g_ids in generated_ids]

for gen, ref in zip(generated_texts, test_references):
    print(f"Generated: {gen}")
    print(f"Reference: {ref}")
    reward = compute_reward(gen, ref)
    print(f"Reward: {reward:.4f}\n")

print("Reward function tested successfully.")

Testing reward function on QA data...


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Generated: 80%
Reference: 80%
BLEU Score: 0.0000
Cosine Similarity: 1.0000
Perplexity Score: 0.4018
Composite Reward: 0.4804
Reward: 0.4804

Generated: ansans
Reference: unanswerable
BLEU Score: 0.0000
Cosine Similarity: 0.2541
Perplexity Score: 0.1090
Composite Reward: 0.1235
Reward: 0.1235

Generated: by our external senses this sensory information is supplied to internals which merge all pieces into whole unified conscious experience this process of perception and abstraction is nex of soul and body for material body may only only objects while immaterial soul may on ->
Reference: external senses
BLEU Score: 0.0000
Cosine Similarity: 0.5592
Perplexity Score: 0.3307
Composite Reward: 0.2898
Reward: 0.2898

Generated: ington
Reference: wellington college
BLEU Score: 0.0000
Cosine Similarity: 0.3615
Perplexity Score: 0.1538
Composite Reward: 0.1753
Reward: 0.1753

Generated: 200 million watt
Reference: 200 million kilowatt hours
BLEU Score: 0.0000
Cosine Similarity: 0.6674
Perplexity S

/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Generated: : start international polska symptoms ->
Reference: warsaw
BLEU Score: 0.0000
Cosine Similarity: 0.2864
Perplexity Score: 0.2135
Composite Reward: 0.1573
Reward: 0.1573

Generated: 
Reference: london
BLEU Score: 0.0000
Cosine Similarity: 0.2501
Perplexity Score: 0.4950
Composite Reward: 0.1990
Reward: 0.1990

Generated: complete the triple triple with the exact object:: manor symptoms ->
Reference: basel
BLEU Score: 0.0000
Cosine Similarity: -0.0178
Perplexity Score: 0.1524
Composite Reward: 0.0233
Reward: 0.0233

Generated: complete the triple triple with the exact object:aeg symptoms
Reference: berlin
BLEU Score: 0.0000
Cosine Similarity: -0.0037
Perplexity Score: 0.1343
Composite Reward: 0.0254
Reward: 0.0254

Generated: complete the triple triple with the exact object:rai symptoms rai
Reference: rome
BLEU Score: 0.0000
Cosine Similarity: 0.0118
Perplexity Score: 0.1034
Composite Reward: 0.0254
Reward: 0.0254

Reward function tested successfully.


In [ ]:
# Define PPO Policy Network and Environment

# Policy Network for DPR (simple MLP to adjust embeddings)
class PolicyNetwork(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=256):
        super(PolicyNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Tanh()  # Output adjustment to [-1, 1]
        )

    def forward(self, x):
        return self.network(x)

# RL Environment for DPR ranking
class RankingEnvironment:
    def __init__(self, ctx_encoder, question_encoder, candidates, candidate_embeddings, val_loader, task: str = "qa"):
        self.ctx_encoder = ctx_encoder
        self.question_encoder = question_encoder
        self.candidates = candidates
        self.candidate_embeddings = candidate_embeddings
        self.val_loader = val_loader
        self.task = task
        self.current_batch = None
        self.current_idx = 0
        self.batch_iterator = iter(val_loader)

    def reset(self):
        try:
            self.current_batch = next(self.batch_iterator)
            self.current_idx = 0
        except StopIteration:
            self.batch_iterator = iter(self.val_loader)
            self.current_batch = next(self.batch_iterator)
            self.current_idx = 0
        question_inputs = {
            "input_ids": self.current_batch["dpr_input_ids"][self.current_idx].to(CONFIG.DEVICE).unsqueeze(0),  # Add batch dimension
            "attention_mask": self.current_batch["dpr_attention_mask"][self.current_idx].to(CONFIG.DEVICE).unsqueeze(0)  # Add batch dimension
        }
        with torch.no_grad():
            state = self.question_encoder(**question_inputs).pooler_output  # Shape: (1, 768)
        return state

    def step(self, action):
        # Action is the adjustment to the question embedding
        question_inputs = {
            "input_ids": self.current_batch["dpr_input_ids"][self.current_idx].to(CONFIG.DEVICE).unsqueeze(0),  # Add batch dimension
            "attention_mask": self.current_batch["dpr_attention_mask"][self.current_idx].to(CONFIG.DEVICE).unsqueeze(0)  # Add batch dimension
        }
        with torch.no_grad():
            question_embedding = self.question_encoder(**question_inputs).pooler_output  # Shape: (1, 768)
        adjusted_embedding = question_embedding + action  # Adjust embedding
        similarities = torch.matmul(adjusted_embedding, self.candidate_embeddings.T)  # Shape: (1, num_candidates)
        rankings = torch.argsort(similarities, dim=1, descending=True)
        ref = self.current_batch["answer"][self.current_idx]
        ref_idx = self.candidates.index(ref) if ref in self.candidates else -1
        done = False
        if ref_idx == -1:
            reward = 0.0
            done = True
        else:
            rank = (rankings == ref_idx).nonzero(as_tuple=True)[0].item() + 1 if ref_idx in rankings else len(self.candidates)
            # Generate text for reward computation
            top_idx = rankings[0, 0].item()
            generated_text = self.candidates[top_idx]
            reference_text = ref
            reward = compute_reward(generated_text, reference_text)
            print(f"Task: {self.task}, Rank: {rank}, Reward: {reward:.4f}")

        # Move to next sample
        self.current_idx += 1
        if self.current_idx >= len(self.current_batch["answer"]):
            try:
                self.current_batch = next(self.batch_iterator)
                self.current_idx = 0
            except StopIteration:
                done = True

        # Get next state
        if not done:
            next_question_inputs = {
                "input_ids": self.current_batch["dpr_input_ids"][self.current_idx].to(CONFIG.DEVICE).unsqueeze(0),  # Add batch dimension
                "attention_mask": self.current_batch["dpr_attention_mask"][self.current_idx].to(CONFIG.DEVICE).unsqueeze(0)  # Add batch dimension
            }
            with torch.no_grad():
                next_state = self.question_encoder(**next_question_inputs).pooler_output
        else:
            next_state = None

        return next_state, reward, done

print("PPO policy network and environment defined.")

PPO policy network and environment defined.


In [ ]:
# Implement PPO for DPR Fine-Tuning on QA and Triple Data

# PPO Implementation
def compute_gae(rewards, values, next_value, gamma=0.99, lam=0.95):
    advantages = []
    gae = 0
    rewards = rewards + [next_value]
    values = values + [next_value]
    for t in reversed(range(len(rewards) - 1)):
        delta = rewards[t] + gamma * values[t + 1] - values[t]
        gae = delta + gamma * lam * gae
        advantages.insert(0, gae)
    return advantages

def ppo_update(policy, optimizer, states, actions, old_log_probs, advantages, returns, clip_epsilon=0.2, epochs=10):
    for _ in range(epochs):
        policy.train()
        new_log_probs = []
        for state, action in zip(states, actions):
            state = state.to(CONFIG.DEVICE)
            action = action.to(CONFIG.DEVICE)
            adjustment = policy(state)
            dist = torch.distributions.Normal(adjustment, 0.1)
            log_prob = dist.log_prob(action).sum(dim=-1)
            new_log_probs.append(log_prob)
        new_log_probs = torch.stack(new_log_probs)

        # Compute ratio
        ratios = torch.exp(new_log_probs - torch.stack(old_log_probs))

        # Compute surrogate loss
        surr1 = ratios * torch.tensor(advantages, device=CONFIG.DEVICE)
        surr2 = torch.clamp(ratios, 1 - clip_epsilon, 1 + clip_epsilon) * torch.tensor(advantages, device=CONFIG.DEVICE)
        policy_loss = -torch.min(surr1, surr2).mean()

        # Update policy
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        print(f"Policy Loss: {policy_loss.item():.4f}")

# Fine-tune DPR with PPO on QA and triple data
policy = PolicyNetwork(input_dim=768, hidden_dim=256).to(CONFIG.DEVICE)
value_network = PolicyNetwork(input_dim=768, hidden_dim=256).to(CONFIG.DEVICE)
policy_optimizer = optim.Adam(policy.parameters(), lr=1e-4)
value_optimizer = optim.Adam(value_network.parameters(), lr=1e-4)

# Create environments for QA and triple tasks
qa_env = RankingEnvironment(ctx_encoder, question_encoder, all_candidates, candidate_embeddings, qa_val_loader, task="qa")
triple_env = RankingEnvironment(ctx_encoder, question_encoder, all_candidates, candidate_embeddings, triple_val_loader, task="triple")

num_episodes = 50  # Number of episodes per task
max_steps_per_episode = 50  # Maximum steps per episode

# Fine-tune on QA data
print("Fine-tuning DPR with PPO on QA data...")
for episode in range(num_episodes):
    print(f"QA Episode {episode + 1}/{num_episodes}")
    state = qa_env.reset()
    # Ensure state has the correct shape (batch_size, embedding_dim)
    if state.dim() == 1:
        state = state.unsqueeze(0)  # Add batch dimension if needed
    states, actions, rewards, old_log_probs, values = [], [], [], [], []
    done = False
    episode_steps = 0

    while not done and episode_steps < max_steps_per_episode:
        state = state.to(CONFIG.DEVICE)
        with torch.no_grad():
            adjustment = policy(state)
            dist = torch.distributions.Normal(adjustment, 0.1)
            action = dist.sample()
            log_prob = dist.log_prob(action).sum(dim=-1)
            value = value_network(state).mean()  # Reduce to scalar by taking the mean

        next_state, reward, done = qa_env.step(action)

        # Ensure next_state has the correct shape
        if next_state is not None and next_state.dim() == 1:
            next_state = next_state.unsqueeze(0)

        states.append(state)
        actions.append(action)
        rewards.append(reward)
        old_log_probs.append(log_prob)
        values.append(value.item())  # Now a scalar

        state = next_state
        episode_steps += 1

    # Compute returns and advantages
    with torch.no_grad():
        next_value = value_network(state).mean().item() if state is not None else 0.0
    advantages = compute_gae(rewards, values, next_value)
    returns = [r + next_value for r in rewards]

    # Update policy and value network
    ppo_update(policy, policy_optimizer, states, actions, old_log_probs, advantages, returns)

    # Update value network
    value_loss = 0
    for state, ret in zip(states, returns):
        state = state.to(CONFIG.DEVICE)
        value = value_network(state).mean()  # Reduce to scalar
        value_loss += (value - ret) ** 2
    value_loss = value_loss / len(states)
    value_optimizer.zero_grad()
    value_loss.backward()
    value_optimizer.step()
    print(f"QA Value Loss: {value_loss.item():.4f}")

# Fine-tune on triple data
print("Fine-tuning DPR with PPO on triple data...")
for episode in range(num_episodes):
    print(f"Triple Episode {episode + 1}/{num_episodes}")
    state = triple_env.reset()
    # Ensure state has the correct shape
    if state.dim() == 1:
        state = state.unsqueeze(0)
    states, actions, rewards, old_log_probs, values = [], [], [], [], []
    done = False
    episode_steps = 0

    while not done and episode_steps < max_steps_per_episode:
        state = state.to(CONFIG.DEVICE)
        with torch.no_grad():
            adjustment = policy(state)
            dist = torch.distributions.Normal(adjustment, 0.1)
            action = dist.sample()
            log_prob = dist.log_prob(action).sum(dim=-1)
            value = value_network(state).mean()  # Reduce to scalar

        next_state, reward, done = triple_env.step(action)

        # Ensure next_state has the correct shape
        if next_state is not None and next_state.dim() == 1:
            next_state = next_state.unsqueeze(0)

        states.append(state)
        actions.append(action)
        rewards.append(reward)
        old_log_probs.append(log_prob)
        values.append(value.item())  # Now a scalar

        state = next_state
        episode_steps += 1

    # Compute returns and advantages
    with torch.no_grad():
        next_value = value_network(state).mean().item() if state is not None else 0.0
    advantages = compute_gae(rewards, values, next_value)
    returns = [r + next_value for r in rewards]

    # Update policy and value network
    ppo_update(policy, policy_optimizer, states, actions, old_log_probs, advantages, returns)

    # Update value network
    value_loss = 0
    for state, ret in zip(states, returns):
        state = state.to(CONFIG.DEVICE)
        value = value_network(state).mean()  # Reduce to scalar
        value_loss += (value - ret) ** 2
    value_loss = value_loss / len(states)
    value_optimizer.zero_grad()
    value_loss.backward()
    value_optimizer.step()
    print(f"Triple Value Loss: {value_loss.item():.4f}")

# Save updated DPR models
ctx_encoder.save_pretrained(os.path.join(CONFIG.BASE_PATH, "dpr_ctx_encoder_rl"))
question_encoder.save_pretrained(os.path.join(CONFIG.BASE_PATH, "dpr_question_encoder_rl"))
print("Saved RL-fine-tuned DPR models.")

Streaming output truncated to the last 5000 lines.
BLEU Score: 0.0000
Cosine Similarity: 0.1217
Perplexity Score: 0.0895
Composite Reward: 0.0666
Task: triple, Rank: 1, Reward: 0.0666
BLEU Score: 0.0000
Cosine Similarity: 0.0905
Perplexity Score: 0.1808
Composite Reward: 0.0724
Task: triple, Rank: 1, Reward: 0.0724
BLEU Score: 0.0000
Cosine Similarity: 0.2703
Perplexity Score: 0.0603
Composite Reward: 0.1202
Task: triple, Rank: 1, Reward: 0.1202
BLEU Score: 0.0000
Cosine Similarity: 0.0862
Perplexity Score: 0.2426
Composite Reward: 0.0830
Task: triple, Rank: 1, Reward: 0.0830
BLEU Score: 0.0000
Cosine Similarity: 0.1784
Perplexity Score: 0.2599
Composite Reward: 0.1233
Task: triple, Rank: 1, Reward: 0.1233
BLEU Score: 0.0000
Cosine Similarity: 0.1747
Perplexity Score: 0.2695
Composite Reward: 0.1238
Task: triple, Rank: 1, Reward: 0.1238
BLEU Score: 0.0000
Cosine Similarity: 0.2737
Perplexity Score: 0.0282
Composite Reward: 0.1151
Task: triple, Rank: 1, Reward: 0.1151
BLEU Score: 0.0000

In [ ]:
# Evaluate DPR After RL Fine-Tuning on QA and Triple Data
import json

# Evaluate DPR on full and small candidate pools for QA
def evaluate_dpr(ctx_encoder, question_encoder, val_loader, candidates, small_candidate_pool: bool = False, task: str = "qa"):
    ctx_encoder.eval()
    question_encoder.eval()
    mrr, precision_at_1 = [], []
    eval_candidates = candidates[:100] if small_candidate_pool else candidates
    print(f"Task: {task}, Using candidate pool size: {len(eval_candidates)}")
    candidate_inputs = ctx_tokenizer(eval_candidates, return_tensors="pt", padding=True, truncation=True, max_length=CONFIG.MAX_LENGTH)
    candidate_inputs = {k: v.to(CONFIG.DEVICE) for k, v in candidate_inputs.items()}
    with torch.no_grad():
        candidate_embeddings = ctx_encoder(**candidate_inputs).pooler_output
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Evaluating {task}"):
            question_inputs = {
                "input_ids": batch["dpr_input_ids"].to(CONFIG.DEVICE),
                "attention_mask": batch["dpr_attention_mask"].to(CONFIG.DEVICE)
            }
            references = batch["answer"]
            question_embeddings = question_encoder(**question_inputs).pooler_output
            similarities = torch.matmul(question_embeddings, candidate_embeddings.T)
            rankings = torch.argsort(similarities, dim=1, descending=True)
            for i, (ranking, ref) in enumerate(zip(rankings, references)):
                ref_idx = eval_candidates.index(ref) if ref in eval_candidates else -1
                if ref_idx == -1:
                    continue
                rank = (ranking == ref_idx).nonzero(as_tuple=True)[0].item() + 1 if ref_idx in ranking else len(eval_candidates)
                mrr.append(1.0 / rank)
                precision_at_1.append(1.0 if rank == 1 else 0.0)
            del question_inputs, similarities, rankings
            torch.cuda.empty_cache()
    avg_mrr = np.mean(mrr)
    avg_precision_at_1 = np.mean(precision_at_1)
    print(f"DPR Evaluation ({task}):")
    print(f"MRR: {avg_mrr:.4f}")
    print(f"Precision@1: {avg_precision_at_1:.4f}")
    return avg_mrr, avg_precision_at_1

# Evaluate on QA data
dpr_mrr_full_qa_rl, dpr_precision_full_qa_rl = evaluate_dpr(ctx_encoder, question_encoder, qa_val_loader, all_candidates, small_candidate_pool=False, task="qa")
dpr_mrr_small_qa_rl, dpr_precision_small_qa_rl = evaluate_dpr(ctx_encoder, question_encoder, qa_val_loader, all_candidates, small_candidate_pool=True, task="qa")

# Evaluate on triple data
dpr_mrr_full_triple_rl, dpr_precision_full_triple_rl = evaluate_dpr(ctx_encoder, question_encoder, triple_val_loader, all_candidates, small_candidate_pool=False, task="triple")
dpr_mrr_small_triple_rl, dpr_precision_small_triple_rl = evaluate_dpr(ctx_encoder, question_encoder, triple_val_loader, all_candidates, small_candidate_pool=True, task="triple")

# Save evaluation results
results = {
    "dpr_full_qa_rl": {"mrr": dpr_mrr_full_qa_rl, "precision_at_1": dpr_precision_full_qa_rl},
    "dpr_small_pool_qa_rl": {"mrr": dpr_mrr_small_qa_rl, "precision_at_1": dpr_precision_small_qa_rl},
    "dpr_full_triple_rl": {"mrr": dpr_mrr_full_triple_rl, "precision_at_1": dpr_precision_full_triple_rl},
    "dpr_small_pool_triple_rl": {"mrr": dpr_mrr_small_triple_rl, "precision_at_1": dpr_precision_small_triple_rl}
}

results_path = os.path.join(CONFIG.BASE_PATH, "rl_evaluation_results.json")
with open(results_path, "w") as f:
    json.dump(results, f)
print(f"Saved RL evaluation results at {results_path}")

Task: qa, Using candidate pool size: 2024


Evaluating qa: 100%|██████████| 7/7 [00:01<00:00,  5.64it/s]


DPR Evaluation (qa):
MRR: 0.4157
Precision@1: 0.3200
Task: qa, Using candidate pool size: 100


Evaluating qa: 100%|██████████| 7/7 [00:00<00:00, 12.54it/s]


DPR Evaluation (qa):
MRR: 0.5214
Precision@1: 0.3750
Task: triple, Using candidate pool size: 2024


Evaluating triple: 100%|██████████| 63/63 [00:04<00:00, 15.08it/s]


DPR Evaluation (triple):
MRR: 0.0149
Precision@1: 0.0070
Task: triple, Using candidate pool size: 100


Evaluating triple: 100%|██████████| 63/63 [00:03<00:00, 19.05it/s]

DPR Evaluation (triple):
MRR: 0.0982
Precision@1: 0.0541
Saved RL evaluation results at /content/drive/MyDrive/LJMU-Datasets/rl_evaluation_results.json
